# TIFF to MP4/GIF Conversion

In [1]:
import napari
import tifffile as tiff
import os
import numpy as np
import dask.array as da
import dask
from dask.delayed import delayed
from napari_animation import Animation
import subprocess

# Set Dask to use multi-threading for better performance
dask.config.set(scheduler="threads")

import subprocess

def speed_up_video_ffmpeg(input_path, speed_factor): # I think this is super duper slow
    """
    Speeds up an MP4 video using FFmpeg with NVIDIA GPU hardware acceleration (NVENC),
    without processing the audio.
    
    :param input_path: Path to the input MP4 file.
    :param speed_factor: Factor by which to speed up the video (e.g., 2.0 for double speed).
    """
    # Adjust the setpts filter for speeding up the video
    video_filter = f"setpts=1/{speed_factor}*PTS"

    # Output path
    filename, file_format = os.path.splitext(input_path)  # Splits into "video" and ".mp4"
    output_path = f"{filename}_speed{speed_factor}{file_format}"  # "video_speed2.mp4"

    # Create the FFmpeg command
    speed_cmd = [
        "ffmpeg", "-hwaccel", "cuda", "-i", input_path, 
        "-vf", video_filter,  # Use -vf instead of -filter:v
        "-c:v", "h264_nvenc", "-preset", "veryfast",  
        "-vsync", "0",  
        "-threads", "8",
        "-an",  
        output_path
    ]

    
    try:
        # Execute the FFmpeg command and capture output
        result = subprocess.run(speed_cmd, check=True, capture_output=True, text=True)
        print(result.stdout)  # Print FFmpeg output
    except subprocess.CalledProcessError as e:
        # Print error output
        print("Error occurred:")
        print(e.stderr)


def convert_tiff_to_video(tiff_path: str, img_name: list[str] = None, file_format: str = 'mp4', video_length: int = 30, fps: int = None,  speed_factor: float = 1.0):
    """
    Converts a folder of TIFF images into a video animation (MP4 or GIF) using Napari.
    
    Parameters:
        tiff_path (str): Base path to the folder containing TIFF images.
        img_name (list of str, optional): List of folder names to process. Defaults to None (processes all).
        file_format (str): Format of the output video ('mp4' or 'gif'). Defaults to 'mp4'.
        video_length (int): Desired length of the video in seconds (automatically adjusts FPS). Only works if fps=None. Defaults to 30.
        fps (int): Frames per second for the output video. Overrides video_length Defaults to None.
        speed_factor (float): Factor to speed up video by. Defaults to 1.0.
    """
    video_path = tiff_path.replace('tiff', 'video')
    os.makedirs(video_path, exist_ok=True)
    
    if img_name is None:
        img_name = [name for name in os.listdir(tiff_path) if os.path.isdir(os.path.join(tiff_path, name))]
    
    for name in img_name:
        folder_path = os.path.join(tiff_path, name)
        save_path = os.path.join(video_path, name)
        os.makedirs(save_path, exist_ok=True)
        output_file = os.path.join(save_path, f"{name}.{file_format}")

        tiff_files = sorted(
            [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith('.tiff')]
        )

        if not tiff_files:
            print(f"Warning: No TIFF files found in {folder_path}. Skipping...")
            continue

        lazy_images = [delayed(tiff.imread)(f) for f in tiff_files]
        image_stack = da.stack([da.from_delayed(img, shape=lazy_images[0].shape, dtype=np.uint8) for img in lazy_images])
        num_frames = image_stack.shape[0]
        if fps is None:
            fps = max(1, round(num_frames / video_length))  # Ensure FPS is at least 1

        viewer = napari.Viewer()
        layer = viewer.add_image(image_stack, colormap='gray', name="TIFF Stack", scale=[1, 1, 1])
        
        animation = Animation(viewer)
        for i in range(num_frames):
            viewer.dims.set_point(0, i)
            animation.capture_keyframe()

        animation.animate(output_file, fps=fps, file_format=file_format)
        print(f"Animation saved at: {output_file} (FPS: {fps})")
        napari.run()

        if file_format == 'mp4' and speed_factor != 1.0:
            speed_up_video_ffmpeg(output_file, speed_factor)

In [2]:
tiff_path = r'D:\Darren\Files\outputs\tiff\pretrained_mic50_tab30_spray60'

convert_tiff_to_video(tiff_path, ['1_Microsphere'], 'mp4', video_length=30)
convert_tiff_to_video(tiff_path, ['3_SprayDriedDispersion'], 'mp4', video_length=30)

TypeError: Delayed objects of unspecified length have no len()